# 02 - Kubeflow Training with Feast Integration

**TrainJob → Feast → PyTorch DDP → MLflow**

## Configuration Options

| Mode | `use_ray` | `get_historical_features()` | Dependencies |
|------|-----------|------------------------------|--------------|
| **Simple** | `false` | File-based (single node) | `feast[postgres]` |
| **Distributed** | `true` | KubeRay (parallel PIT joins) | `feast[ray,postgres]`, `codeflare-sdk` |

```
┌─────────────────────────────────────────────────────────────┐
│                     TRAINJOB POD                            │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  training_df = store.get_historical_features(               │
│      entity_df, features=training_features                  │
│  ).to_df()                                                  │
│           │                                                 │
│           ├── use_ray=false → File offline store (local)    │
│           │                                                 │
│           └── use_ray=true  → KubeRay cluster (distributed) │
│                                      │                      │
└──────────────────────────────────────│──────────────────────┘
                                       ▼
                          ┌─────────────────────┐
                          │   KUBERAY CLUSTER   │
                          │  (if use_ray=true)  │
                          └─────────────────────┘
```


In [ ]:
%pip install -q kubeflow-training "mlflow>=3.0" yamlmagic
%load_ext yamlmagic


## Configuration


In [ ]:
%%yaml parameters

# =============================================================================
# 🔀 MODE SELECTION - Must match 01-feast-features.ipynb
# =============================================================================
use_ray: true  # true = KubeRay for distributed training, false = file-based only

# =============================================================================
# Cluster
# =============================================================================
namespace: feast-trainer-demo
shared_pvc: feast-pvc
runtime: torch-distributed  # Use built-in supported runtime (PVC added via PodTemplateOverrides)

# =============================================================================
# KubeRay Configuration (only used if use_ray: true)
# =============================================================================
kuberay:
  cluster_name: feast-ray
  skip_tls: true

# =============================================================================
# Training Hyperparameters
# =============================================================================
epochs: 30
batch_size: 64
learning_rate: 0.0005
weight_decay: 0.001

# =============================================================================
# Model Architecture
# =============================================================================
model:
  hidden_dims: [512, 256, 128, 64]
  dropout: 0.3

# =============================================================================
# Distributed Training
# =============================================================================
num_workers: 1
resources:
  cpu: 4
  memory: 8Gi
gpu_type: nvidia  # none, nvidia, amd
gpu_count: 1

# =============================================================================
# MLflow
# =============================================================================
mlflow_experiment: sales-forecasting

# =============================================================================
# Paths (PVC mounted at /shared)
# =============================================================================
paths:
  data: /shared/data
  models: /shared/models
  feature_repo: /shared/feature_repo

In [ ]:
# Derived
parameters['mlflow_uri'] = f"http://mlflow.{parameters['namespace']}.svc.cluster.local:5000"


## Authentication


In [ ]:
import os

K8S_TOKEN = os.getenv("K8S_TOKEN", "<YOUR_TOKEN>")
K8S_API_SERVER = os.getenv("K8S_API_SERVER", "<YOUR_API_SERVER>")


In [ ]:
from kubernetes import client as k8s
from kubeflow.training import TrainerClient, CustomTrainer
from kubeflow.training.types import KubernetesBackendConfig
from kubeflow.training.types import (
    PodTemplateOverrides, PodTemplateOverride,
    PodSpecOverride, ContainerOverride,
    Labels, Annotations
)

cfg = k8s.Configuration()
if K8S_TOKEN and K8S_API_SERVER:
    cfg.host = K8S_API_SERVER
    cfg.verify_ssl = False
    cfg.api_key = {"authorization": f"Bearer {K8S_TOKEN}"}

trainer_client = TrainerClient(
    KubernetesBackendConfig(
        namespace=parameters['namespace'],
        client_configuration=cfg if K8S_TOKEN else None
    )
)


## Training Runtime


In [ ]:
runtime = trainer_client.get_runtime(parameters['runtime'])

## Training Function


In [ ]:
def train_sales_model(parameters):
    """
    Distributed training with FEAST INTEGRATION.
    
    KEY PATTERN: This function runs INSIDE the TrainJob pod and calls
    Feast directly to get training data. This ensures feature consistency
    between training and inference.
    
    Flow:
    1. Load Feast FeatureStore from shared PVC
    2. Call get_historical_features() with training_features service
    3. Train PyTorch model with DDP
    4. Log to MLflow
    """
    import os
    import json
    import numpy as np
    import torch
    import torch.nn as nn
    import torch.distributed as dist
    from torch.nn.parallel import DistributedDataParallel as DDP
    from torch.utils.data import DataLoader, Dataset, DistributedSampler
    import pandas as pd
    from sklearn.preprocessing import StandardScaler
    from pathlib import Path
    from datetime import datetime, timezone, timedelta
    import joblib

    # =========================================================================
    # Device Setup
    # =========================================================================
    backend = 'nccl' if torch.cuda.is_available() else 'gloo'
    dist.init_process_group(backend=backend)
    rank, world_size = dist.get_rank(), dist.get_world_size()
    local_rank = int(os.environ.get("LOCAL_RANK", 0))
    device = torch.device(f"cuda:{local_rank}" if torch.cuda.is_available() else "cpu")
    
    if rank == 0:
        print(f"DDP: rank={rank}/{world_size}, device={device}")

    # =========================================================================
    # Configuration
    # =========================================================================
    mlflow_uri = os.getenv("MLFLOW_TRACKING_URI", "http://mlflow:5000")
    data_dir = parameters.get('paths', {}).get('data', '/shared/data')
    model_dir = parameters.get('paths', {}).get('models', '/shared/models')
    feature_repo = parameters.get('paths', {}).get('feature_repo', '/shared/feature_repo')
    epochs = parameters.get('epochs', 30)
    batch_size = parameters.get('batch_size', 64)
    lr = parameters.get('learning_rate', 0.0005)
    weight_decay = parameters.get('weight_decay', 0.001)
    hidden_dims = parameters.get('model', {}).get('hidden_dims', [512, 256, 128, 64])
    dropout = parameters.get('model', {}).get('dropout', 0.3)

    # =========================================================================
    # Model Definition
    # =========================================================================
    class SalesMLP(nn.Module):
        def __init__(self, input_dim, hidden_dims, dropout):
            super().__init__()
            layers = []
            prev_dim = input_dim
            for h_dim in hidden_dims:
                layers.extend([
                    nn.Linear(prev_dim, h_dim),
                    nn.BatchNorm1d(h_dim),
                    nn.ReLU(),
                    nn.Dropout(dropout),
                ])
                prev_dim = h_dim
            layers.append(nn.Linear(prev_dim, 1))
            self.net = nn.Sequential(*layers)
        
        def forward(self, x):
            return self.net(x).squeeze(-1)
    
    class SalesDataset(Dataset):
        def __init__(self, X, y):
            self.X = torch.tensor(X, dtype=torch.float32)
            self.y = torch.tensor(y, dtype=torch.float32)
        def __len__(self): return len(self.X)
        def __getitem__(self, i): return self.X[i], self.y[i]

    # =========================================================================
    # MLflow Setup (rank 0)
    # =========================================================================
    if rank == 0:
        import mlflow
        mlflow.set_tracking_uri(mlflow_uri)
        mlflow.set_experiment(parameters.get('mlflow_experiment', 'sales-forecasting'))
        run = mlflow.start_run(run_name=f"train-{datetime.now().strftime('%Y%m%d-%H%M%S')}")
        print(f"MLflow tracking URI: {mlflow_uri}")
        print(f"MLflow run ID: {run.info.run_id}")

    # =========================================================================
    # FEAST: get_historical_features() - File-based or KubeRay (distributed)
    # =========================================================================
    if rank == 0:
        os.makedirs(model_dir, exist_ok=True)
        use_ray = parameters.get('use_ray', False)
        
        print("=" * 60)
        if use_ray:
            print("🍕 FEAST + KUBERAY: Distributed get_historical_features()")
        else:
            print("🍕 FEAST: File-based get_historical_features()")
        print("=" * 60)
        
        # Setup Ray configuration (only if use_ray=true)
        if use_ray:
            os.environ["FEAST_RAY_USE_KUBERAY"] = "true"
            os.environ["FEAST_RAY_CLUSTER_NAME"] = parameters.get('kuberay', {}).get('cluster_name', 'feast-ray')
            os.environ["FEAST_RAY_NAMESPACE"] = parameters.get('namespace', 'feast-trainer-demo')
            os.environ["FEAST_RAY_SKIP_TLS"] = str(parameters.get('kuberay', {}).get('skip_tls', True)).lower()
            
            # In-cluster auth for CodeFlare SDK
            sa_token_path = "/var/run/secrets/kubernetes.io/serviceaccount/token"
            if os.path.exists(sa_token_path):
                with open(sa_token_path) as f:
                    os.environ["FEAST_RAY_AUTH_TOKEN"] = f.read()
                os.environ["FEAST_RAY_AUTH_SERVER"] = f"https://{os.environ.get('KUBERNETES_SERVICE_HOST')}:{os.environ.get('KUBERNETES_SERVICE_PORT')}"
                print("🔐 CodeFlare SDK auth configured (in-cluster)")
            
            # Copy Ray config to feature_store.yaml (same pattern as manifest)
            import shutil
            ray_config = f"{feature_repo}/feature_store_ray.yaml"
            target_config = f"{feature_repo}/feature_store.yaml"
            if os.path.exists(ray_config):
                shutil.copy(ray_config, target_config)
                print(f"📄 Using Ray config: {ray_config}")
        else:
            print("ℹ️  Using file-based offline store (no KubeRay)")
        
        from feast import FeatureStore
        store = FeatureStore(repo_path=feature_repo)
        print(f"Loaded Feast from: {feature_repo}")
        
        # Create entity DataFrame
        entity_rows = []
        base_date = datetime(2010, 2, 5, tzinfo=timezone.utc)  # Walmart dataset start
        NUM_WEEKS, NUM_STORES, NUM_DEPTS = 104, 10, 5
        
        for week in range(NUM_WEEKS):
            event_ts = base_date + timedelta(weeks=week)
            for store_id in range(1, NUM_STORES + 1):
                for dept_id in range(1, NUM_DEPTS + 1):
                    entity_rows.append({"store_id": store_id, "dept_id": dept_id, "event_timestamp": event_ts})
        
        entity_df = pd.DataFrame(entity_rows)
        print(f"Entity DataFrame: {len(entity_df):,} rows")
        
        # =====================================================================
        # KEY CALL: get_historical_features() with Feature Service
        # This retrieves the SAME features that inference will use
        # =====================================================================
        import time
        start_time = time.time()
        
        feature_service = store.get_feature_service("training_features")
        print(f"Using Feature Service: {feature_service.name}")
        
        training_data = store.get_historical_features(
            entity_df=entity_df,
            features=feature_service,
        )
        training_df = training_data.to_df()
        elapsed = time.time() - start_time
        
        print(f"✅ Feast returned {len(training_df):,} rows in {elapsed:.2f}s")
        print(f"   Throughput: {len(entity_df)/elapsed:.0f} rows/sec")
        
        # Temporal split (80/20)
        training_df = training_df.dropna(subset=["weekly_sales"])
        training_df = training_df.sort_values("event_timestamp")
        split_idx = int(len(training_df) * 0.8)
        train_df = training_df.iloc[:split_idx]
        val_df = training_df.iloc[split_idx:]
        
        print(f"Train: {len(train_df):,}, Val: {len(val_df):,}")
        
        # Feature columns (exclude identifiers and target)
        exclude_cols = ["store_id", "dept_id", "date", "event_timestamp", "weekly_sales"]
        feature_cols = [c for c in training_df.columns 
                        if c not in exclude_cols
                        and training_df[c].dtype in [np.float64, np.int64, np.float32, np.int32]]
        
        print(f"Feature columns ({len(feature_cols)}): {feature_cols}")
        
        X_train = train_df[feature_cols].fillna(0).values
        y_train = train_df["weekly_sales"].values
        X_val = val_df[feature_cols].fillna(0).values
        y_val = val_df["weekly_sales"].values
        
        # Scale features
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)
        
        y_scaler = StandardScaler()
        y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
        y_val_scaled = y_scaler.transform(y_val.reshape(-1, 1)).flatten()
        
        # Save scalers
        joblib.dump(scaler, f"{model_dir}/scaler.pkl")
        joblib.dump(y_scaler, f"{model_dir}/y_scaler.pkl")
        joblib.dump(feature_cols, f"{model_dir}/feature_cols.pkl")
        joblib.dump({"scaler_X": scaler, "scaler_y": y_scaler}, f"{model_dir}/scalers.joblib")
        
        # Save for other ranks
        np.save(f"{model_dir}/.X_train.npy", X_train)
        np.save(f"{model_dir}/.y_train.npy", y_train_scaled)
        np.save(f"{model_dir}/.X_val.npy", X_val)
        np.save(f"{model_dir}/.y_val.npy", y_val_scaled)
        np.save(f"{model_dir}/.y_val_orig.npy", y_val)
        np.save(f"{model_dir}/.input_dim.npy", np.array([X_train.shape[1]]))
        
        # Log to MLflow
        mlflow.log_params({
            "epochs": epochs, "batch_size": batch_size, "learning_rate": lr,
            "weight_decay": weight_decay, "hidden_dims": str(hidden_dims),
            "dropout": dropout, "world_size": world_size,
            "train_rows": len(train_df), "val_rows": len(val_df),
            "input_dim": X_train.shape[1], "feature_service": "training_features",
        })
        
        print("Data loading and preprocessing complete")
    
    dist.barrier()
    
    # Load data on all ranks
    X_train = np.load(f"{model_dir}/.X_train.npy")
    y_train = np.load(f"{model_dir}/.y_train.npy")
    X_val = np.load(f"{model_dir}/.X_val.npy")
    y_val = np.load(f"{model_dir}/.y_val.npy")
    y_val_orig = np.load(f"{model_dir}/.y_val_orig.npy")
    input_dim = int(np.load(f"{model_dir}/.input_dim.npy")[0])
    
    dist.barrier()

    # =========================================================================
    # Training
    # =========================================================================
    print(f"[Rank {rank}] Training...")
    
    train_ds = SalesDataset(X_train, y_train)
    val_ds = SalesDataset(X_val, y_val)
    sampler = DistributedSampler(train_ds, num_replicas=world_size, rank=rank)
    train_loader = DataLoader(train_ds, batch_size=batch_size, sampler=sampler, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2)
    
    model = SalesMLP(input_dim, hidden_dims, dropout).to(device)
    model = DDP(model, device_ids=[local_rank] if torch.cuda.is_available() else None)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=3)
    criterion = nn.MSELoss()
    
    best_val_loss = float('inf')
    best_mape = float('inf')
    
    for epoch in range(epochs):
        sampler.set_epoch(epoch)
        model.train()
        train_loss = 0.0
        
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0.0
        predictions, actuals = [], []
        
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                output = model(X_batch.to(device))
                val_loss += criterion(output, y_batch.to(device)).item()
                predictions.extend(output.cpu().numpy())
                actuals.extend(y_batch.numpy())
        
        val_loss /= len(val_loader)
        scheduler.step(val_loss)
        
        if rank == 0:
            predictions = np.array(predictions)
            y_scaler = joblib.load(f"{model_dir}/y_scaler.pkl")
            preds_orig = y_scaler.inverse_transform(predictions.reshape(-1, 1)).flatten()
            
            mask = np.abs(y_val_orig) > 1000
            mape = np.mean(np.abs((y_val_orig[mask] - preds_orig[mask]) / y_val_orig[mask])) * 100 if mask.sum() > 0 else np.nan
            rmse = np.sqrt(np.mean((y_val_orig - preds_orig) ** 2))
            mae = np.mean(np.abs(y_val_orig - preds_orig))
            
            mlflow.log_metrics({
                "train_loss": train_loss, "val_loss": val_loss,
                "mape": mape, "rmse": rmse, "mae": mae,
                "learning_rate": optimizer.param_groups[0]['lr'],
            }, step=epoch)
            
            print(f"Epoch {epoch+1}/{epochs} | Train: {train_loss:.4f} | Val: {val_loss:.4f} | MAPE: {mape:.1f}% | RMSE: {rmse:.0f}")
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_mape = mape
                torch.save(model.module.state_dict(), f"{model_dir}/best_model.pt")
                print("  → Saved best model")
        
        dist.barrier()
    
    # =========================================================================
    # Save Model (rank 0)
    # =========================================================================
    dist.barrier()
    
    if rank == 0:
        mlflow.log_metrics({"best_val_loss": best_val_loss, "best_mape": best_mape})
        
        # Load best model for logging
        best_model = SalesMLP(input_dim, hidden_dims, dropout)
        best_model.load_state_dict(torch.load(f"{model_dir}/best_model.pt"))
        best_model.eval()
        
        # Log model to MLflow
        model_info = mlflow.pytorch.log_model(
            best_model, artifact_path="model",
            pip_requirements=["torch>=2.0", "numpy", "scikit-learn"],
        )
        
        # Register model
        try:
            model_uri = f"runs:/{mlflow.active_run().info.run_id}/model"
            mlflow.register_model(model_uri, "sales-forecasting-mlp")
            print("Model registered in MLflow Model Registry")
        except Exception as e:
            print(f"Model registration skipped: {e}")
        
        # Log artifacts
        mlflow.log_artifact(f"{model_dir}/scaler.pkl", artifact_path="preprocessing")
        mlflow.log_artifact(f"{model_dir}/y_scaler.pkl", artifact_path="preprocessing")
        mlflow.log_artifact(f"{model_dir}/feature_cols.pkl", artifact_path="preprocessing")
        
        # Model metadata
        with open(f"{model_dir}/model_metadata.json", "w") as f:
            json.dump({
                "model_type": "SalesMLP", "input_dim": input_dim,
                "hidden_dims": hidden_dims, "dropout": dropout,
                "best_mape": float(best_mape), "best_val_loss": float(best_val_loss),
            }, f, indent=2)
        mlflow.log_artifact(f"{model_dir}/model_metadata.json")
        
        mlflow.end_run()
        
        # Cleanup temp files
        for f in [".X_train.npy", ".y_train.npy", ".X_val.npy", ".y_val.npy", ".y_val_orig.npy", ".input_dim.npy"]:
            try: os.remove(f"{model_dir}/{f}")
            except: pass
        
        print("=" * 60)
        print(f"✅ TRAINING COMPLETE!")
        print(f"   Best val_loss: {best_val_loss:.4f}")
        print(f"   Best MAPE: {best_mape:.1f}%")
        print(f"   Model saved to: {model_dir}/best_model.pt")
        print("=" * 60)
    
    dist.barrier()
    dist.destroy_process_group()

## Submit Training Job


In [ ]:
from datetime import datetime

job_id = datetime.now().strftime("%m%d-%H%M")
job_name = f"sales-training-{job_id}"


In [ ]:
# GPU resources
resources = {**parameters['resources']}
if parameters['gpu_type'] == 'nvidia' and parameters['gpu_count'] > 0:
    resources["nvidia.com/gpu"] = parameters['gpu_count']
elif parameters['gpu_type'] == 'amd' and parameters['gpu_count'] > 0:
    resources["amd.com/gpu"] = parameters['gpu_count']

# Packages: include Ray dependencies only if use_ray is true
base_packages = ["scikit-learn", "pandas", "pyarrow", "joblib", "mlflow>=3.0"]
if parameters.get('use_ray', False):
    packages = ["feast[ray,postgres]==0.59.0", "codeflare-sdk"] + base_packages
    description = f"Sales forecasting with Feast+KubeRay - {job_id}"
else:
    packages = ["feast[postgres]==0.59.0"] + base_packages
    description = f"Sales forecasting with Feast (file-based) - {job_id}"

# Submit training job
job = trainer_client.train(
    trainer=CustomTrainer(
        func=train_sales_model,
        num_nodes=parameters['num_workers'],
        resources_per_node=resources,
        packages_to_install=packages,
        env={
            "MLFLOW_TRACKING_URI": parameters['mlflow_uri'],
            "RUN_NAME": f"sales-forecast-{job_id}",
        },
    ),
    runtime=runtime,
    parameters=parameters,
    options=[
        Labels({"app": "sales-forecasting", "run-id": job_id}),
        Annotations({"description": description}),
        PodTemplateOverrides(
            PodTemplateOverride(
                target_jobs=["node"],
                spec=PodSpecOverride(
                    volumes=[{"name": "shared", "persistentVolumeClaim": {"claimName": parameters['shared_pvc']}}],
                    containers=[ContainerOverride(name="node", volume_mounts=[{"name": "shared", "mountPath": "/shared"}])]
                )
            )
        ),
    ],
)

## Monitor Progress


In [ ]:
trainer_client.wait_for_job_status(name=job, status={"Running"}, timeout=300)


In [ ]:
_ = trainer_client.get_job_logs(name=job, follow=True)


In [ ]:
trainer_client.wait_for_job_status(name=job, status={"Complete", "Failed"}, timeout=3600)


## MLflow


In [ ]:
import mlflow

mlflow.set_tracking_uri(parameters['mlflow_uri'])
experiment = mlflow.get_experiment_by_name(parameters['mlflow_experiment'])
runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id], max_results=5, order_by=["start_time DESC"])
runs[["tags.mlflow.runName", "metrics.best_mape"]]


## Cleanup


In [ ]:
# trainer_client.delete_job(name=job)
